<a href="https://colab.research.google.com/github/pmxfa/sp-shapely/blob/main/sp_timevae_exchange.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install synthcity

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 503.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 552.2/552.2 kB 38.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of torchtext to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
INFO: pip is st

# Training

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import sys
import warnings
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

import synthcity.logger as log
from synthcity.plugins import Plugins
from synthcity.plugins.core.dataloader import TimeSeriesDataLoader
from synthcity.utils.serialization import save_to_file, load_from_file

log.add(sink=sys.stderr, level="INFO")

Mounted at /content/drive
[KeOps] Compiling cuda jit compiler engine ... OK
[pyKeOps] Compiling nvrtc binder for python ... OK


In [ ]:
# Define file path
file_path = "/content/drive/Shareddrives/sp_env/datasets/Exchange Rate/exchange_rate.txt"

df = pd.read_csv(file_path)
print(df.head())
print(df.info())
print(df.isnull().sum())

   0.785500  1.611000  0.861698  0.634196  0.211242  0.006838  0.593000  \
0    0.7818    1.6100  0.861104  0.633513  0.211242  0.006863    0.5940   
1    0.7867    1.6293  0.861030  0.648508  0.211242  0.006975    0.5973   
2    0.7860    1.6370  0.862069  0.650618  0.211242  0.006953    0.5970   
3    0.7849    1.6530  0.861995  0.656254  0.211242  0.006940    0.5985   
4    0.7866    1.6537  0.861030  0.654879  0.211242  0.006887    0.6040   

   0.525486  
0  0.523972  
1  0.526316  
2  0.523834  
3  0.527426  
4  0.526177  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7587 entries, 0 to 7586
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   0.785500  7587 non-null   float64
 1   1.611000  7587 non-null   float64
 2   0.861698  7587 non-null   float64
 3   0.634196  7587 non-null   float64
 4   0.211242  7587 non-null   float64
 5   0.006838  7587 non-null   float64
 6   0.593000  7587 non-null   float64
 7   0.5

In [ ]:
# for exchange dataset only
df.columns = [
    'Australia', 'British_Pound', 'Canada', 'Switzerland',
    'China', 'Japan', 'New_Zealand', 'Singapore'
]

# set seed and randomly choose 2 features
np.random.seed(42)

# Filter the dataframe to only include selected features
# df_latest = df_latest[selected_features]
selected_features = np.random.choice(df.columns, size=2, replace=False)
print(f"Selected features: {selected_features}")

# Immediately filter your full DataFrame down to just those two series:
df = df[selected_features]

# Set the index as a daily date range starting from 1990-01-01
df.index = pd.date_range(start='1990-01-01', periods=len(df), freq='D')

# Optional: show the first few rows
print(df.head())

Selected features: ['British_Pound' 'Japan']
            British_Pound     Japan
1990-01-01         1.6100  0.006863
1990-01-02         1.6293  0.006975
1990-01-03         1.6370  0.006953
1990-01-04         1.6530  0.006940
1990-01-05         1.6537  0.006887


In [ ]:
print(len(df))

7587


In [ ]:
# Keep the latest 5000 rows
df_latest = df.tail(5000)

# Train-test split: 70% for training (for TimeGAN), 30% for testing (TSTR)
train_size = int(0.7 * len(df_latest))
df_train = df_latest.iloc[:train_size]
df_test = df_latest.iloc[train_size:]  # use later for LSTM-TSTR

# Normalize the data
scaler = MinMaxScaler()
scaled_train = scaler.fit_transform(df_train)
df_scaled_train = pd.DataFrame(scaled_train, columns=df_train.columns, index=df_train.index)
scaled_test = scaler.transform(df_test)
df_scaled_test = pd.DataFrame(scaled_train, columns=df_train.columns, index=df_train.index)


# set sequence length to 30 to capture monthly patterns
# dataset = daily
sequence_length = 30
temporal_data = []
observation_times = []

for start in range(len(df_scaled_train) - sequence_length + 1):
    sequence = df_scaled_train.iloc[start:start + sequence_length].reset_index(drop=True)
    temporal_data.append(sequence)
    observation_times.append(list(range(sequence_length)))  # relative time within the window


# Dummy outcome for data loader
dummy_outcome = pd.DataFrame(np.zeros(len(temporal_data)), columns=["outcome"])

# --- Create DataLoader for TimeGAN ---
loader = TimeSeriesDataLoader(
    temporal_data=temporal_data,  # List of sequences (DataFrames)
    observation_times=observation_times,  # List of time indices (DataFrames)
    static_data=None,  # No static data for now (can be set if needed)
    outcome=dummy_outcome,  # Dummy outcome for forecasting
)

# Print the loader info
print(f"TimeSeriesDataLoader created with {len(temporal_data)} sequences")

TimeSeriesDataLoader created with 3471 sequences


In [ ]:
print(len(df_scaled))  # Check the length of the dataframe
print(loader.dataframe())

5000
        seq_id  seq_time_id  seq_temporal_British_Pound  seq_temporal_Japan  \
0            0            0                   -0.730340           -0.278606   
1            0            1                   -0.721990           -0.302775   
2            0            2                   -0.758753           -0.323491   
3            0            3                   -0.781029           -0.328325   
4            0            4                   -0.770740           -0.312443   
...        ...          ...                         ...                 ...   
104125    3470           25                   -0.113456            1.939406   
104126    3470           26                   -0.107151            1.912475   
104127    3470           27                   -0.166367            1.871042   
104128    3470           28                   -0.169654            1.873804   
104129    3470           29                   -0.196819            1.906950   

        seq_out_outcome  
0                   

All available hyperparameters are listed below:

model = plugin_timegan.TimeGAN( n_iter=500, generator_n_layers_hidden=3, generator_n_units_hidden=100, generator_nonlin="relu", generator_dropout=0.1, discriminator_n_layers_hidden=3, discriminator_n_units_hidden=100, discriminator_nonlin="leaky_relu", discriminator_dropout=0.1, discriminator_n_iter=2, lr=0.0002, weight_decay=0.0001, batch_size=200, encoder_max_clusters=10, mode="LSTM", gamma_penalty=10.0, moments_penalty=10.0, embedding_penalty=5.0 )

In [ ]:
syn_model = Plugins().get("timevae")

[2025-04-18T05:29:55.860824+0000][1486][CRITICAL] module disabled: /usr/local/lib/python3.11/dist-packages/synthcity/plugins/generic/plugin_goggle.py
[2025-04-18T05:29:55.860824+0000][1486][CRITICAL] module disabled: /usr/local/lib/python3.11/dist-packages/synthcity/plugins/generic/plugin_goggle.py


In [ ]:
# Print all parameters of initialized model
for attr in dir(syn_model):
    if not attr.startswith("_") and not callable(getattr(syn_model, attr)):
        print(f"{attr}: {getattr(syn_model, attr)}")

batch_size: 64
class_name: TimeVAEPlugin
clipping_value: 0
compress_dataset: False
decoder_batch_norm: False
decoder_dropout: 0.01
decoder_n_layers_hidden: 2
decoder_n_units_hidden: 150
decoder_nonlin: leaky_relu
decoder_nonlin_out_continuous: tanh
decoder_nonlin_out_discrete: softmax
decoder_residual: True
device: cuda
embedding_penalty: 10
encoder: None
encoder_batch_norm: False
encoder_dropout: 0.1
encoder_max_clusters: 20
encoder_n_layers_hidden: 3
encoder_n_units_hidden: 300
encoder_nonlin: leaky_relu
expecting_conditional: False
fitted: False
gamma_penalty: 1
lr: 0.001
mode: LSTM
module_name: synthcity.plugins.time_series.plugin_timevae
module_relative_path: ../time_series/plugin_timevae.py
moments_penalty: 100
n_iter: 1000
n_iter_print: 10
outcome_encoder: TabularEncoder(cat_encoder_params={'handle_unknown': 'ignore',
                                   'sparse_output': False},
               categorical_encoder='onehot',
               cont_encoder_params={'n_components': 20},
 

## fitting the model

In [ ]:
print(loader.shape)

# Train the model
syn_model.fit(loader)

save_to_file('/content/drive/Shareddrives/sp_env/saved_models/VAE_Exchange.pkl', syn_model)

In [ ]:
# --- Generate Synthetic Data ---
n_samples = len(temporal_data)
syn_data = syn_model.generate(count=n_samples)
print(syn_data.shape)

(104130, 5)


In [ ]:
# --- Save with automated format ---
import datetime
import os
# Get the current date and time
now = datetime.datetime.now()
timestamp = now.strftime("%m%d%y-%H%M%S")  # MMDDYY-HHMMSS format

# Define the base directory
base_dir = "/content/drive/Shareddrives/sp_env/synthetic_datasets/TimeVAE/exchange"  #CHANGE THIS
if not os.path.exists(base_dir):
    os.makedirs(base_dir)

# Construct the filename
model_name = type(syn_model).__name__.lower() # Get model name dynamically
filename = f"{timestamp}-{model_name}-3500.csv"
filepath = os.path.join(base_dir, filename)

# Save the data
df_syn = syn_data.dataframe()
df_syn.to_csv(filepath, index=False)

print(f"Synthetic data saved to: {filepath}")

Synthetic data saved to: /content/drive/Shareddrives/sp_env/synthetic_datasets/TimeVAE/exchange/041825-055628-timevaeplugin-3500.csv


# Evaluation

## Prerequisites

In [ ]:
selected_columns = ['seq_temporal_British_Pound','seq_temporal_Japan']
# Ensure real_data and synthetic_data only contain the selected columns
real_data = loader.dataframe()[selected_columns].to_numpy()
synthetic_data = syn_data.dataframe()[selected_columns].to_numpy()

In [ ]:
print(real_data, "\n ------------------------------------------------------- \n", synthetic_data)
print(type(real_data),type(synthetic_data))
print(real_data.shape,synthetic_data.shape)

[[-0.7303401  -0.27860632]
 [-0.72199044 -0.30277519]
 [-0.75875348 -0.32349137]
 ...
 [-0.16636713  1.87104216]
 [-0.16965352  1.87380432]
 [-0.19681919  1.9069502 ]] 
 ------------------------------------------------------- 
 [[ 2.20882183  2.18614607]
 [ 0.00475425  0.94483251]
 [-0.67864349  1.74184096]
 ...
 [ 0.38989849 -0.42228482]
 [ 0.38989849  0.70548638]
 [-0.50453688  0.70548638]]
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(104130, 2) (104130, 2)


## Generate distance metrics

### Helper Functions

In [ ]:
from scipy.stats import wasserstein_distance, entropy
import numpy as np

def compute_wasserstein(real_data, synthetic_data, selected_columns):
    """
    Computes Wasserstein Distance between real and synthetic time-series data.

    """

    # Ensure both datasets have the same number of samples
    min_length = min(len(real_data), len(synthetic_data))
    real_trimmed = real_data[:min_length]  # Keep original order (no random sampling)
    synthetic_trimmed = synthetic_data[:min_length]  # Match size
    print(real_trimmed.shape,synthetic_trimmed.shape)

    wasserstein_results = {}

    # Compute Wasserstein Distance for each feature
    for i, col in enumerate(selected_columns):
        w_dist = wasserstein_distance(real_trimmed[:, i], synthetic_trimmed[:, i])
        wasserstein_results[col] = w_dist
        print(f"{w_dist}")

    return wasserstein_results

def compute_kl_divergence(real_data, synthetic_data, selected_columns, bins=50):
    """
    Computes KL Divergence between real and synthetic time-series data.

    """

    # Ensure both datasets have the same number of samples
    min_length = min(len(real_data), len(synthetic_data))
    real_trimmed = real_data[:min_length]  # Keep original order
    synthetic_trimmed = synthetic_data[:min_length]  # Match size

    kl_results = {}

    for i, col in enumerate(selected_columns):
        # Compute histogram-based probability distributions
        real_hist, _ = np.histogram(real_trimmed[:, i], bins=bins, density=True)
        synth_hist, _ = np.histogram(synthetic_trimmed[:, i], bins=bins, density=True)

        # Avoid zero probabilities (KL Divergence is undefined for zero values)
        real_hist += 1e-10
        synth_hist += 1e-10

        # Compute KL Divergence
        kl_div = entropy(real_hist, synth_hist)
        kl_results[col] = kl_div
        print(f"{kl_div}")

    return kl_results

### Generate Metrics

In [ ]:
wasserstein_results = compute_wasserstein(real_data, synthetic_data, selected_columns)
kl_results = compute_kl_divergence(real_data, synthetic_data, selected_columns)

print("Wasserstein Distance Results:")
print(wasserstein_results)

print("KL Divergence Results:")
print(kl_results)

(104130, 2) (104130, 2)
0.08810128254045688
0.25819928742749454
Wasserstein Distance Results:
{'seq_temporal_British_Pound': 0.08810128254045688, 'seq_temporal_Japan': 0.25819928742749454}
13.230451545175983
10.598849017183563
KL Divergence Results:
{'seq_temporal_British_Pound': 13.230451545175983, 'seq_temporal_Japan': 10.598849017183563}


# LSTM downstream

In [ ]:
from google.colab import drive
drive.mount('/content/drive')